In [94]:
import pandas as pd
import ujson
import pickle
import numpy as np

In [95]:
fpath = '../data/SFPD.csv'
pddf = pd.read_csv(fpath)
pddf.head(1)

,IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location,PdId
0,160000108,ASSAULT,BATTERY,Thursday,12/31/2015,23:58,SOUTHERN,"ARREST, BOOKED",800 Block of BRYANT ST,-122.403405,37.775421,"(37.775420706711, -122.403404791479)",16000010804134


In [96]:
hpdf = pd.read_csv('../data/SFlistingsN.csv')
hpdf.head()

,Unnamed: 0,averageListingPrice,medianListingPrice,numberOfProperties,type,weekEndingDate,neighborhoodName,TruliaNID,City,State
0,0,1120143,699000,7,All Properties,2009-07-04,Alamo Square,1386,San Francisco,CA
1,1,551000,535000,3,1 Bedroom Properties,2009-07-04,Alamo Square,1386,San Francisco,CA
2,2,553667,543000,3,1 Bedroom Properties,2009-07-04,Alamo Square,1386,San Francisco,CA
3,3,699000,699000,1,2 Bedroom Properties,2009-07-04,Alamo Square,1386,San Francisco,CA
4,4,799000,799000,1,3 Bedroom Properties,2009-07-04,Alamo Square,1386,San Francisco,CA


In [97]:
nbhoods = hpdf['neighborhoodName'].values
nbh = set(nbhoods)
print nbh

set(["Fisherman's Wharf", 'Ingleside Terrace', 'Financial District', 'Forest Knolls', 'West Portal', 'Golden Gate Heights', 'Cow Hollow', 'Bernal Heights', 'Buena Vista Park', 'Treasure Island', 'Duboce Triangle', 'Visitacion Valley', 'Marina', 'Merced Manor', 'Mount Davidson Manor', 'Saint Francis Wood', 'Forest Hill', 'Mission Bay', 'Chinatown', 'Outer Sunset', 'Stonestown', 'Outer Richmond', 'Cole Valley', 'Haight', 'South Beach', 'Portola', 'Sunnyside', 'Miraloma', 'Silver Terrace', 'Golden Gate Park', 'Lower Pacific Heights', 'Lake Street', 'Downtown', 'Lincoln Park - Lobos', 'Crocker Amazon', "Bernt's Knoll", 'Telegraph Hill', 'Inner Sunset', 'Outer Mission', 'Central Richmond', 'Twin Peaks', 'Mission Dolores', 'Inner Richmond', 'Japantown', 'Glen Park', 'Mission Terrace', 'Balboa Terrace', 'Hayes Valley', 'Merced Heights', 'Ingleside Heights', 'Nob Hill', 'Ashbury Heights', 'SoMa', 'Diamond Heights', 'Noe Valley', 'North Beach', 'Sherwood Forest', 'Bayview', 'Civic Center', 'Hun

In [98]:
pddf.replace(to_replace=['FORGERY/COUNTERFEITING','DRUG/NARCOTIC','PORNOGRAPHY/OBSCENE MAT','LARCENY/THEFT'], value=['FORGERY-COUNTERFEITING','DRUG-NARCOTIC','PORNOGRAPHY-OBSCENE MAT','LARCENY-THEFT'], inplace=True, limit=None, regex=False, method='pad', axis=None)
ctype = pddf['Category'].values
ctp = set(ctype)
print ctp

set(['KIDNAPPING', 'WEAPON LAWS', 'SECONDARY CODES', 'WARRANTS', 'PROSTITUTION', 'EMBEZZLEMENT', 'SEX OFFENSES, NON FORCIBLE', 'SUICIDE', 'DRIVING UNDER THE INFLUENCE', 'ROBBERY', 'BURGLARY', 'SUSPICIOUS OCC', 'FAMILY OFFENSES', 'DRUG-NARCOTIC', 'BAD CHECKS', 'DRUNKENNESS', 'GAMBLING', 'OTHER OFFENSES', 'LARCENY-THEFT', 'RECOVERED VEHICLE', 'FRAUD', 'ARSON', 'SEX OFFENSES, FORCIBLE', 'FORGERY-COUNTERFEITING', 'PORNOGRAPHY-OBSCENE MAT', 'TRESPASS', 'LOITERING', 'VANDALISM', 'NON-CRIMINAL', 'EXTORTION', 'LIQUOR LAWS', 'ASSAULT', 'TREA', 'VEHICLE THEFT', 'STOLEN PROPERTY', 'BRIBERY', 'MISSING PERSON', 'DISORDERLY CONDUCT', 'RUNAWAY'])


In [99]:
## Clean Trulia dataset

#only use data for 2 bedroom poperties
hpdf = hpdf[hpdf['type'] == '2 Bedroom Properties']

#make east SF neighborhoods into SFPD districts, those are the ones that map decently/semi-easily with Trulia data. Get rid of the others
Bayview = ['Bayview', 'Central Waterfront', 'Hunters Point', 'Mission Bay', 'Portola', 'Potrero Hill', 'Silver Terrace']
Central = ['Central', 'Downtown', 'Financial District', 'Nob Hill', 'North Beach', 'Russian Hill', 'Telegraph Hill']
Mission = ['Mission', 'Mission Dolores', 'Noe Valley']
Northern = ['Northern', 'Alamo Square', 'Cow Hollow', 'Duboce Triangle', 'Hayes Valley', 'Lower Pacific Heights', 'Marina', 'Pacific Heights', 'Western Addition']
Southern = ['Southern', 'SoMa', 'South Beach']
Tenderloin = ['Tenderloin', 'Civic Center']

districts = [Bayview, Central, Mission, Northern, Southern, Tenderloin]

hpd = pd.DataFrame()

for district in districts:
    hpdnh = pd.DataFrame()
    
    for nh in district:
        hnh=hpdf[hpdf['neighborhoodName'] == nh]
        hnh['SFPD District'] = str(district[0]).upper()
        hpdnh=pd.concat([hpdnh,hnh])

    hpd=pd.concat([hpd,hpdnh])

#drop other columns
hpd.drop('Unnamed: 0', axis=1, inplace=True)
hpd.drop('medianListingPrice', axis=1, inplace=True)
hpd.drop('numberOfProperties', axis=1, inplace=True)
hpd.drop('TruliaNID', axis=1, inplace=True)

#average together multiple listings from the same week
hpd = hpd.set_index(pd.to_datetime(hpd['weekEndingDate']))
hpd = hpd.groupby(['weekEndingDate','SFPD District']).mean()
hpd = pd.DataFrame(hpd).reset_index()
hpd = hpd.set_index(pd.to_datetime(hpd['weekEndingDate']))
hpd.drop('weekEndingDate', axis=1, inplace=True)

/opt/conda/lib/python2.7/site-packages/IPython/kernel/__main__.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [100]:
#Fix Northern NoRthern in HPD
rsfl = ['12M','6M', '3M','1M']

dist_list = ['Bayview','Central','Mission','Northern','SOUTHERN','TENDERLOIN']

ctypes = sorted(['WEAPON LAWS','SECONDARY CODES','WARRANTS','ROBBERY','BURGLARY',
	 'SUSPICIOUS OCC','ASSAULT','DRUNKENNESS','OTHER OFFENSES','FRAUD','DRUG-NARCOTIC',
	 'TRESPASS', 'LARCENY-THEFT','VANDALISM','NON-CRIMINAL','VEHICLE THEFT','MISSING PERSON',
	 'DISORDERLY CONDUCT'])

In [113]:
crime = 'Drug-Narcotic'
rsf = "6M"
pdi = pddf[pddf['Category'] != None]
pdct = pd.crosstab(pdi['Date'],pdi['PdDistrict'])
pdct = pdct.set_index(pd.to_datetime(pdct.index))
pdct = pdct.resample(rsf, how='sum')
pdct.head()

PdDistrict,BAYVIEW,CENTRAL,INGLESIDE,MISSION,NORTHERN,PARK,RICHMOND,SOUTHERN,TARAVAL,TENDERLOIN
2003-01-31,1531,1120,1232,1882,1605,699,666,2116,947,1158
2003-07-31,7829,6775,6913,10405,9741,3980,3841,13089,5733,6275
2004-01-31,7809,6937,7122,10798,9368,4226,3823,12449,5719,6474
2004-07-31,8356,7025,7339,10812,9623,4065,4064,12609,6037,6059
2005-01-31,7385,6691,6640,9982,8866,4170,3661,12089,5985,5944


In [104]:
for rsf in rsfl:
    #create Housing for every dist table
    hpi = hpd[['averageListingPrice']]
    hpi = hpd.reset_index().pivot(index='weekEndingDate', columns='SFPD District', values='averageListingPrice')
    hpct = hpi.set_index(pd.to_datetime(hpi.index))
    hpct = hpct.resample(rsf, how='mean')
    hpct['ALL'] = hpct.mean(axis=1)
    hpct.drop(hpct.head(1).index, inplace=True)
    hpct.drop(hpct.tail(1).index, inplace=True)
    print hpct.head()
    fname = '../cast2/app/static/data/AllDistHouse-{}.csv'.format(rsf.upper())
    hpct.to_csv(fname)
print 'Data Exported'

SFPD District         BAYVIEW         CENTRAL         MISSION        NORTHERN  \
weekEndingDate                                                                  
2010-07-31      628558.663301  1119735.120575   701207.482385   809409.899660   
2011-07-31      570413.282292  1145566.865199   688610.761973   779037.545544   
2012-07-31      627592.466958  1289804.675028   659886.666163   828653.380643   
2013-07-31      614038.790036  1200211.269313   737936.628770   997427.010951   
2014-07-31      826171.353952  1486577.289493  1017294.494434  1096965.178627   

SFPD District         SOUTHERN     TENDERLOIN             ALL  
weekEndingDate                                                 
2010-07-31       952733.849105  672822.053863   814077.844815  
2011-07-31      1010053.028846  619562.808013   802207.381978  
2012-07-31      1134141.308333  693766.766026   872307.543858  
2013-07-31      1375173.942308  736797.759615   943597.566832  
2014-07-31      1506591.941346  838359.686275  1

In [107]:
#Create Crime Data Tables
for rsf in rsfl:
    for crime in ctypes:
        pdi = pddf[pddf['Category'] == crime.upper()]
        pdct = pd.crosstab(pdi['Date'],pdi['PdDistrict'])
        pdct = pdct.set_index(pd.to_datetime(pdct.index))
        pdct = pdct.resample(rsf, how='sum')
        pdct['ALL'] = pdct.sum(axis=1)
        pdct.drop(pdct.head(1).index, inplace=True)
        pdct.drop(pdct.tail(1).index, inplace=True)
        fname = '../cast2/app/static/data/perc/{}-{}.csv'.format(crime.upper(), rsf.upper())
        pdct.to_csv(fname)
        print '%s %s Data Exported' % (crime.upper(), rsf.upper())  
    print 'All %s Data Exported' % (rsf.upper()) 
print 'All Data Exported'

ASSAULT 12M Data Exported
BURGLARY 12M Data Exported
DISORDERLY CONDUCT 12M Data Exported
DRUG-NARCOTIC 12M Data Exported
DRUNKENNESS 12M Data Exported
FRAUD 12M Data Exported
LARCENY-THEFT 12M Data Exported
MISSING PERSON 12M Data Exported
NON-CRIMINAL 12M Data Exported
OTHER OFFENSES 12M Data Exported
ROBBERY 12M Data Exported
SECONDARY CODES 12M Data Exported
SUSPICIOUS OCC 12M Data Exported
TRESPASS 12M Data Exported
VANDALISM 12M Data Exported
VEHICLE THEFT 12M Data Exported
WARRANTS 12M Data Exported
WEAPON LAWS 12M Data Exported
All 12M Data Exported
ASSAULT 6M Data Exported
BURGLARY 6M Data Exported
DISORDERLY CONDUCT 6M Data Exported
DRUG-NARCOTIC 6M Data Exported
DRUNKENNESS 6M Data Exported
FRAUD 6M Data Exported
LARCENY-THEFT 6M Data Exported
MISSING PERSON 6M Data Exported
NON-CRIMINAL 6M Data Exported
OTHER OFFENSES 6M Data Exported
ROBBERY 6M Data Exported
SECONDARY CODES 6M Data Exported
SUSPICIOUS OCC 6M Data Exported
TRESPASS 6M Data Exported
VANDALISM 6M Data Exporte

TypeError: not all arguments converted during string formatting

In [116]:
#Create Crime Data Tables
for rsf in rsfl:
        pdi = pddf[pddf['Category'] != None]
        pdct = pd.crosstab(pdi['Date'],pdi['PdDistrict'])
        pdct = pdct.set_index(pd.to_datetime(pdct.index))
        pdct = pdct.resample(rsf, how='sum')
        pdct['ALL'] = pdct.sum(axis=1)
        pdct.drop(pdct.head(1).index, inplace=True)
        pdct.drop(pdct.tail(1).index, inplace=True)
        fname = '../cast2/app/static/data/perc/{}-{}.csv'.format('ALL', rsf.upper())
        pdct.to_csv(fname) 
        print 'All %s Data Exported' % (rsf.upper()) 
print 'All Data Exported'

All 12M Data Exported
All 6M Data Exported
All 3M Data Exported
All 1M Data Exported
All Data Exported


In [106]:
pdct.head()

PdDistrict,BAYVIEW,CENTRAL,INGLESIDE,MISSION,NORTHERN,PARK,RICHMOND,SOUTHERN,TARAVAL,TENDERLOIN,ALL
2003-02-28,17,9,17,12,10,6,5,19,11,16,12.2
2003-03-31,19,6,14,12,12,7,10,10,15,7,11.2
2003-04-30,17,2,22,10,9,9,10,13,8,9,10.9
2003-05-31,21,2,12,16,5,5,4,8,5,13,9.1
2003-06-30,14,4,15,15,5,5,7,8,7,6,8.6
